In [ ]:
from transaction_history_processor import PortfolioHistory
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
from IPython.display import display, HTML
import plotly.graph_objects as go

In [ ]:
if not os.path.exists('portfolio_history.json'):
    print("portfolio_history.json not found. Generating it now...")
    portfolio_history = PortfolioHistory()
    portfolio_history.process_transaction_history(
        input_file='transaction_history.json',
        save_output=True,
        output_file='portfolio_history.json'
    )
    print("portfolio_history.json has been generated.")

with open('portfolio_history.json', 'r') as f:
    data = json.load(f)
    sectors = data['sectors']
    portfolio_data = data['portfolios']

In [ ]:
def plot_stock_proportions(portfolio_data, date=None, show_values=True, main_threshold=3.0):
    if date is None:
        date = max(portfolio_data.keys())

    if date not in portfolio_data:
        print(f"No data available for date: {date}")
        return

    holdings = portfolio_data[date]['holdings']
    labels = list(holdings.keys())
    values = [data['quantity'] * data['market_price']
              for data in holdings.values()]

    total = sum(values)
    percentages = [(value / total) * 100 for value in values]

    main_holdings = list(zip(labels, values, percentages))
    main_holdings.sort(key=lambda x: x[2], reverse=True)

    other_holdings = [h for h in main_holdings if h[2] < main_threshold]
    if len(other_holdings) > 1:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
        others_value = sum(v for _, v, _ in other_holdings)
        others_percentage = sum(p for _, _, p in other_holdings)
        main_holdings = [h for h in main_holdings if h[2] >=
                         main_threshold] + [("Others", others_value, others_percentage)]
    else:
        fig, ax1 = plt.subplots(figsize=(10, 10))
        ax2 = None

    main_labels, main_values, main_percentages = zip(*main_holdings)
    wedges1, texts1, autotexts1 = ax1.pie(main_percentages, labels=main_labels, autopct='%1.1f%%',
                                          startangle=90, wedgeprops={'linewidth': 0.5, 'edgecolor': 'white'})

    if show_values:
        main_legend = [f'{label} (${value:,.2f})' for label, value in zip(
            main_labels, main_values)]
    else:
        main_legend = main_labels

    ax1.legend(wedges1, main_legend, title="Holdings",
               loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
    ax1.set_title(f'Stock Proportions by Value on {date}')

    if ax2:
        other_labels, other_values, other_percentages = zip(*other_holdings)
        wedges2, texts2, autotexts2 = ax2.pie(other_percentages, labels=other_labels, autopct='%1.1f%%',
                                              startangle=90, wedgeprops={'linewidth': 0.5, 'edgecolor': 'white'})

        if show_values:
            other_legend = [f'{label} (${value:,.2f})' for label, value in zip(
                other_labels, other_values)]
        else:
            other_legend = other_labels

        ax2.legend(wedges2, other_legend, title="Other Holdings",
                   loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
        ax2.set_title(f'Breakdown of Other Holdings on {date}')

    plt.tight_layout()
    plt.show()


plot_stock_proportions(portfolio_data, show_values=False)

In [ ]:
# plot_stock_proportions(
#     portfolio_history, date="2023-10-26")

In [ ]:
def plot_sector_proportions(portfolio_data, sectors, date=None):
    if date is None:
        date = max(portfolio_data.keys())

    if date not in portfolio_data:
        print(f"No data available for date: {date}")
        return

    holdings = portfolio_data[date]['holdings']
    sector_counts = {}
    sector_symbols = {}
    for symbol, data in holdings.items():
        sector = sectors.get(symbol, 'ETF')
        if sector not in sector_counts:
            sector_counts[sector] = 0
            sector_symbols[sector] = []
        sector_counts[sector] += data['quantity'] * data['market_price']
        sector_symbols[sector].append(symbol)

    labels = list(sector_counts.keys())
    sizes = list(sector_counts.values())
    total = sum(sizes)
    percentages = [(size / total) * 100 for size in sizes]

    fig, ax = plt.subplots(figsize=(10, 7))
    wedges, texts, autotexts = ax.pie(sizes, autopct='%1.1f%%', startangle=140,
                                      wedgeprops={'linewidth': 0.5,
                                                  'edgecolor': 'white'},
                                      textprops={'fontsize': 8})
    plt.title(f'Sector Proportions on {date}')
    plt.axis('equal')

    for i, wedge in enumerate(wedges):
        sector = labels[i]
        symbols = sector_symbols[sector]
        symbols_text = ", ".join(symbols)

        angle = (wedge.theta2 - wedge.theta1) / 2. + wedge.theta1
        x = np.cos(np.radians(angle))
        y = np.sin(np.radians(angle))

        horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
        connectionstyle = f"angle,angleA=0,angleB={angle}"

        ax.annotate(symbols_text, xy=(x, y), xytext=(1.35 * np.sign(x), 1.4 * y),
                    horizontalalignment=horizontalalignment, fontsize=8,
                    arrowprops=dict(arrowstyle="-", connectionstyle=connectionstyle))

    plt.legend(wedges, labels, title="Sectors",
               loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
    plt.show()
# Call the function with the loaded data
plot_sector_proportions(portfolio_data, sectors)

In [ ]:
import yfinance as yf
def plot_portfolio_growth(portfolio_data, plot_sp500=True):
    # Create dataframe with dates and unrealized gain/loss percentages
    dates = sorted(portfolio_data.keys())
    unrealized_gains = [portfolio_data[d]
                        ['unrealized_gain_loss_percentage'] for d in dates]
    df_plot = pd.DataFrame({'Date': pd.to_datetime(
        dates), 'Unrealized Gain/Loss (%)': unrealized_gains})

    # Create the plot
    fig = go.Figure()

    # Add the area plot for portfolio
    fig.add_trace(go.Scatter(
        x=df_plot['Date'],
        y=df_plot['Unrealized Gain/Loss (%)'],
        fill='tozeroy',
        fillcolor='rgba(255,0,0,0.2)',
        line=dict(color='red', width=2),
        name='Portfolio'
    ))

    if plot_sp500:
        # Fetch S&P 500 data (you'll need to implement this function)
        sp500_data = fetch_sp500_data(
            df_plot['Date'].min(), df_plot['Date'].max())

        # Calculate S&P 500 performance
        sp500_performance = (
            (sp500_data['Close'] - sp500_data['Close'].iloc[0]) / sp500_data['Close'].iloc[0]) * 100

        # Add S&P 500 line to the plot
        fig.add_trace(go.Scatter(
            x=sp500_data.index,
            y=sp500_performance,
            line=dict(color='blue', width=2),
            name='S&P 500'
        ))

    # Add horizontal line at 0%
    fig.add_shape(type="line", x0=df_plot['Date'].min(), x1=df_plot['Date'].max(), y0=0, y1=0,
                  line=dict(color="gray", width=1, dash="dash"))

    # Customize the layout
    fig.update_layout(
        title='Portfolio Performance vs S&P 500',
        xaxis_title='Date',
        yaxis_title='Performance (%)',
        yaxis_tickformat='.2f',
        hovermode='x unified',
        showlegend=True,
        plot_bgcolor='rgba(240, 240, 240, 0.8)',  # Light gray background
        yaxis=dict(
            gridcolor='white',
            zerolinecolor='gray',
            tickformat='.2f',
            ticksuffix='%',
            range=[min(df_plot['Unrealized Gain/Loss (%)'].min(), sp500_performance.min()) - 5,
                   max(df_plot['Unrealized Gain/Loss (%)'].max(), sp500_performance.max()) + 5]
        ),
        xaxis=dict(
            gridcolor='white',
            rangeslider=dict(visible=False),
            rangeselector=dict(
                buttons=list([
                    dict(count=5, label="5D", step="day", stepmode="backward"),
                    dict(count=1, label="1M", step="month", stepmode="backward"),
                    dict(count=6, label="6M", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1Y", step="year", stepmode="backward"),
                    dict(step="all", label="All")
                ])
            )
        )
    )

    # Show the plot
    fig.show()


def fetch_sp500_data(start_date, end_date):
    # Download S&P 500 data
    sp500 = yf.Ticker("^GSPC")
    sp500_data = sp500.history(start=start_date, end=end_date)

    # Return only the 'Close' prices
    return sp500_data[['Close']]

In [ ]:
def analyze_portfolio(portfolio_data, date=None, sort_by='Symbol', ascending=True):
    # If no date is specified, use the latest date
    if date is None:
        date = max(portfolio_data.keys())
    elif date not in portfolio_data:
        raise ValueError(f"No data available for date: {date}")

    data = portfolio_data[date]

    # Prepare data for DataFrame
    holdings_data = []

    for symbol, info in data['holdings'].items():
        holdings_data.append({
            'Symbol': symbol,
            'Quantity': info['quantity'],
            'Unit Cost': info['unit_cost'],
            'Market Price': info['market_price'],
            'Total Cost': info['total_cost'],
            'Market Value': info['market_value'],
            'Unrealized G/L': f"${info['unrealized_gain_loss']:,.2f} ({info['unrealized_gain_loss_percentage']:.2f}%)",
            'Daily Gain': f"${info['daily_gain']:,.2f} ({info['daily_return']*100:.2f}%)"
        })

    # Create DataFrame
    df = pd.DataFrame(holdings_data)

    # Sort the DataFrame
    df = df.sort_values(by=sort_by, ascending=ascending)

    # Format DataFrame
    df['Quantity'] = df['Quantity'].map('{:,.4f}'.format)
    df['Unit Cost'] = df['Unit Cost'].map('${:,.2f}'.format)
    df['Market Price'] = df['Market Price'].map('${:,.2f}'.format)
    df['Total Cost'] = df['Total Cost'].map('${:,.2f}'.format)
    df['Market Value'] = df['Market Value'].map('${:,.2f}'.format)

    # Display the results
    display(HTML(f"<h2>Portfolio Status as of {date}</h2>"))
    display(HTML(df.to_html(index=False, classes='dataframe')))

    print(f"\nPortfolio Summary:")
    print(f"Cash: ${data['cash']:,.2f}")
    print(f"Total Market Value: ${data['total_market_value']:,.2f}")
    print(f"Invested Value: ${data['invested_value']:,.2f}")
    print(f"Unrealized Gain/Loss: ${data['unrealized_gain_loss']
          :,.2f} ({data['unrealized_gain_loss_percentage']:.2f}%)")
    print(f"Realized Gains: ${data['realized_gains']:,.2f}")
    print(f"Total Gain/Loss: ${data['total_gain_loss']:,.2f}")
    print(f"Daily Gain: ${data['daily_gain']
          :,.2f} ({data['daily_return']*100:.2f}%)")

In [ ]:
plot_portfolio_growth(portfolio_data)

In [ ]:
analyze_portfolio(portfolio_data)

In [ ]:
analyze_portfolio(portfolio_data, date="2024-04-10")